In [3]:
!pip install plotly

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
import pandas as pd
import numpy as np

import plotly.express as px

ModuleNotFoundError: No module named 'plotly'

In [25]:
# import dataset 
bmore_pop_data = pd.read_csv('../baltimore_population_2000_2023.csv')

In [26]:
# check the info in the datasets 
bmore_pop_data.info

<bound method DataFrame.info of     Year Population Year on Year Change Change in Percent
0   2000    648,746                   -                 -
1   2001    638,700             -10,046            -1.55%
2   2002    630,367              -8,333            -1.30%
3   2003    623,567              -6,800            -1.08%
4   2004    614,564              -9,003            -1.44%
5   2005    610,068              -4,496            -0.73%
6   2006    607,864              -2,204            -0.36%
7   2007    606,006              -1,858            -0.31%
8   2008    603,758              -2,248            -0.37%
9   2009    601,984              -1,774            -0.29%
10  2010    620,942              18,958             3.15%
11  2011    620,493                -449            -0.07%
12  2012    623,035               2,542             0.41%
13  2013    622,591                -444            -0.07%
14  2014    623,833               1,242             0.20%
15  2015    622,831              -1,002 

In [27]:
# Check data types of columns
bmore_pop_data.dtypes

Year                    int64
Population             object
Year on Year Change    object
Change in Percent      object
dtype: object

In [35]:
# Check for duplicates in the dataset
bmore_pop_data.duplicated().sum()

0

In [28]:
# rename columns for referencing 

bmore_pop_data = bmore_pop_data.rename(columns={"Year": "year", 
                                                "Population": "population", 
                                                "Year on Year Change": "year_on_year_change", 
                                                "Change in Percent": "change_in_percent"})

Changes to Population column

* Replace non numerical values and convert column values to numeric values for calculations 

In [29]:
# remove commas from pop values so they can be converted from objs to ints
bmore_pop_data['population'] = bmore_pop_data['population'].str.replace(',', '')

# convert population values to integer values 
bmore_pop_data['population'] = bmore_pop_data['population'].astype('int')

bmore_pop_data['population']

0     648746
1     638700
2     630367
3     623567
4     614564
5     610068
6     607864
7     606006
8     603758
9     601984
10    620942
11    620493
12    623035
13    622591
14    623833
15    622831
16    616542
17    610853
18    603241
19    594601
20    583157
21    576578
22    569107
23    565239
Name: population, dtype: int64

Changes to Year change column

* Replace non numerical values and convert column values to numeric values for calculations 

In [30]:
# remove "-" from first row 
bmore_pop_data['year_on_year_change'] = bmore_pop_data['year_on_year_change'].replace('-', "0")

# remove commas from pop values so they can be converted from objs to ints
bmore_pop_data['year_on_year_change'] = bmore_pop_data['year_on_year_change'].str.replace(',', '')

# convert population values to integer values 
bmore_pop_data['year_on_year_change'] = bmore_pop_data['year_on_year_change'].astype('int')

bmore_pop_data['year_on_year_change']

0         0
1    -10046
2     -8333
3     -6800
4     -9003
5     -4496
6     -2204
7     -1858
8     -2248
9     -1774
10    18958
11     -449
12     2542
13     -444
14     1242
15    -1002
16    -6289
17    -5689
18    -7612
19    -8640
20   -11444
21    -6579
22    -7471
23    -3868
Name: year_on_year_change, dtype: int64

Changes to Percent change column

* Replace non numerical values and convert column values to numeric values for calculations 
* convert to percentages to floats and then to decimal form 

In [33]:
# remove "-" from first row 
bmore_pop_data['change_in_percent'] = bmore_pop_data['change_in_percent'].replace('-', "0")

# Remove the '%' symbol
bmore_pop_data['change_in_percent'] = bmore_pop_data['change_in_percent'].str.replace('%', '')

# Convert to float and divide by 100 to get the decimal representation
bmore_pop_data['change_in_percent'] = bmore_pop_data['change_in_percent'].astype(float) / 100

bmore_pop_data['change_in_percent']

0     0.0000
1    -0.0155
2    -0.0130
3    -0.0108
4    -0.0144
5    -0.0073
6    -0.0036
7    -0.0031
8    -0.0037
9    -0.0029
10    0.0315
11   -0.0007
12    0.0041
13   -0.0007
14    0.0020
15   -0.0016
16   -0.0101
17   -0.0092
18   -0.0125
19   -0.0143
20   -0.0192
21   -0.0113
22   -0.0130
23   -0.0068
Name: change_in_percent, dtype: float64